This file will take all the locations that have been identified through the "Plants of the World Online" database for the Lythrum genus. Since not all locations are single countries (e.g. 'Czechia-Slovakia', 'Corse', 'Chile North', ...) we first map these occurrencies to single countries, and then for all single countries we see how many distinctions we have.

In [2]:
import pandas as pd
from llama_cpp import Llama
from tqdm import tqdm

In [50]:
#FILES THAT MUST BE PRESENT
plant_distribution_path = './support_files/plant_distribution.csv'
MODEL_PATH = "../guido_TRY_trait_analysis/models/mistral-7b-instruct-v0.1.Q4_K_M.gguf"

#created within the code
regions_mapped_path = './support_files/regions_mapped.csv'
regions_mapped_corrected_path = './support_files/regions_mapped_corrected.csv'


In [36]:
def obtain_regions(plant_distribution_path: str) -> list: 

    """From the dataframe with the plant distribution obtain the complete list of regions"""

    plant_distribution = pd.read_csv(plant_distribution_path, converters={'introduced' : pd.eval, 'native' : pd.eval})

    regions = []

    for i, row in plant_distribution.iterrows():
        if row['introduced'] != []:
            regions += row['introduced']
        if row['native'] != []:
            regions += row['native']

    return sorted(list(set(regions)))

# regions = obtain_regions(plant_distribution_path)

# print("Regions: ", regions)
# print(len(regions))

In [33]:
# llm = Llama(model_path=MODEL_PATH, verbose=False, chat_format='llama-2')

# # Simple test prompt
# response = llm("Q: What is the country or countries that contain the following region: Azores\nA:", max_tokens=32)

# # print(response)

# print(response["choices"][0]["text"])

In [37]:
def build_user_content(region: str)->str:
    """Build the prompt for the user to be passed to the LLM"""

    return ("Given the following region provide the country it belongs to.\n" 
            "Provide a direct answer.\n"
            "Your answer must contain only the name of the country, no other words.\n"
            "If instead the region spans across more than one country, provide the list of countries separated by a comma.\n"
            "Do not write any other word other than the name of the country or countries.\n"
            "Correct answer example: \'Japan\'\n"
            "Correct answer example: \'Russia\'\n"
            "Incorrect answer example: \'Amur is a region in Russia\'\n"
            "Again, only respond with the country name or names separated by a comma.\n"
            f"Region: {region}"
            )

In [45]:
def map_regions(regions: list, model_path: str) -> pd.DataFrame:

    llm = Llama(model_path=MODEL_PATH, verbose=False, chat_format='llama-2')

    mapped_data = []

    for region in tqdm(regions):

        messages = [
                {"role": "system", "content": ("You are a helpful assistant. When given a region, provide the country it belongs to."
                 "Write only the name of the country and nothing else (e.g. \'Japan\')."
                "If the region spans across more than one country, provide the list of countries separated by a comma. "
                "Do not include any additional text or explainations other than the name of the country or the list of countries.")},
                {"role": "user", "content": None}
            ]
        content = build_user_content(region)
        messages[1]['content'] = content

        try:
        #tried switching the temperature (0.2 originally) since I cannot get it to give me the answers in the desired format.
            response = llm.create_chat_completion(messages, temperature=0.05)
            answer = response["choices"][0]["message"]["content"].strip()

            print("\nANSWER: ", answer)

            countries = [c.strip(' .;,\'') for c in answer.split(',') if c.strip()]
            if len(countries) == 1:
                countries = countries[0]

            mapped_data.append({'original': region, 'mapped': countries})

        except Exception as e:
            print(f'Error processing region {region}: {e}')
            mapped_data.append({'original': region, 'mapped': 'ERROR'})

    regions_mapped = pd.DataFrame(mapped_data)

    return regions_mapped


In [ ]:
regions_mapped_df = map_regions(obtain_regions(plant_distribution_path), MODEL_PATH)

regions_mapped_df.to_csv(regions_mapped_path)

regions_mapped_df

llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
  1%|          | 1/195 [00:27<1:29:58, 27.83s/it]


ANSWER:  Afghanistan


  1%|          | 2/195 [00:29<39:09, 12.17s/it]  


ANSWER:  United States


  2%|▏         | 3/195 [00:30<22:45,  7.11s/it]

...

 99%|█████████▉| 194/195 [07:08<00:02,  2.21s/it]


ANSWER:  China


100%|██████████| 195/195 [07:10<00:00,  2.21s/it]


ANSWER:  Yemen


,original,mapped
0,Afghanistan,Afghanistan
1,Alabama,United States
2,Alaska,United States
3,Albania,Albania
4,Alberta,Canada
...,...,...
190,Western Australia,Australia
191,Wisconsin,United States
192,Wyoming,United States
193,Xinjiang,China


In [53]:
#Note that the LLM was not able to recognize few instances of the regions. We manually correct those.
#e.g. for Kriti (row 83) it answered "Kriti is not a recognized region. Could you please provide a different region?"
#for Montana it remained 'Montana'
#for Rhode Island (row 145) it answered "Rhode Island is a state in the United States"
#For Turkey-in-Europe (row 176) it remained 'Turkey-in-Europe'

#I only modify the two rows below, because they are the only two obvious errors. We leave the others, hoping they are present
#in the OpenStreetMaps address

regions_mapped_df_corrected = pd.read_csv(regions_mapped_path)[['original', 'mapped']]

regions_mapped_df_corrected.at[83, 'mapped'] = 'Greece'
regions_mapped_df_corrected.at[145, 'mapped'] = 'United States'

regions_mapped_df_corrected.to_csv(regions_mapped_corrected_path, index=False)



Now. You are going to call the OpenStreetMaps API with the coordinates from the file that the professor will send you.
For each coordinate, you will obtain a json object that is going to have a field called address with a lot of stuff inside. Check if any of that stuff maps to any of the regions mapped or original.

Check your notes for more info. God be with you.